In [19]:
import pandas as pd
import numpy as np

train = pd.read_csv('cleaned_dataset.csv')

train.Ntm_Speciality_Bucket.unique()

train['Ntm_Speciality_Bucket'] = train['Ntm_Speciality_Bucket'].replace({'OB/GYN/Others/PCP/Unknown': 0, 'Endo/Onc/Uro': 1 ,'Rheum':2})

train.Change_T_Score.unique()

train['Change_T_Score'] = train['Change_T_Score'].replace({'No change': 0, 'Unknown': 1 ,'Worsened':2, 'Improved':3})

train.Region.unique()

train['Region'] = train['Region'].replace({'West': 0, 'Midwest': 1 ,'South':2, 'Northeast':3})

train.Race.unique()

train['Race'] = train['Race'].replace({'Caucasian': 0, 'Asian': 1 ,'African American':2})

gender = pd.get_dummies(train['Gender'],drop_first=True)
adherent = pd.get_dummies(train['Adherent_Flag'],drop_first=True)
persistent = pd.get_dummies(train['Persistency_Flag'],drop_first=True)
ethnicity = pd.get_dummies(train['Ethnicity'],drop_first=True)
specialist = pd.get_dummies(train['Ntm_Specialist_Flag'],drop_first=True)

train.drop(['Gender','Adherent_Flag','Persistency_Flag','Ethnicity','Region','Ptid','Ntm_Speciality','Ntm_Specialist_Flag' ],axis=1,inplace=True)

train = pd.concat([train,gender,adherent,persistent,ethnicity,specialist],axis=1)

train.info()

from sklearn.model_selection import train_test_split

X = train.drop('Persistent',axis=1)
y = train['Persistent']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3424 entries, 0 to 3423
Data columns (total 63 columns):
 #   Column                                                              Non-Null Count  Dtype
---  ------                                                              --------------  -----
 0   Race                                                                3424 non-null   int64
 1   Age_Bucket                                                          3424 non-null   int64
 2   Ntm_Speciality_Bucket                                               3424 non-null   int64
 3   Gluco_Record_Prior_Ntm                                              3424 non-null   int64
 4   Gluco_Record_During_Rx                                              3424 non-null   int64
 5   Dexa_Freq_During_Rx                                                 3424 non-null   int64
 6   Dexa_During_Rx                                                      3424 non-null   int64
 7   Frag_Frac_Prior_Ntm              

In [44]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from genetic_selection import GeneticSelectionCV
from sklearn.svm import SVC

In [21]:
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=1000, learning_rate=1.0)

In [55]:
estimator = DecisionTreeClassifier()

In [56]:
model = GeneticSelectionCV(
    estimator, cv=5, verbose=0,
    scoring="f1", max_features=20,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=50,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)

In [67]:
X_train.reset_index(drop=True, inplace=True)

In [68]:
model.fit(X_train, y_train)

GeneticSelectionCV(caching=True, crossover_independent_proba=0.5, cv=5,
                   estimator=DecisionTreeClassifier(), max_features=20,
                   mutation_independent_proba=0.04, n_gen_no_change=10,
                   n_generations=50, n_jobs=-1, n_population=100, scoring='f1')

In [69]:
print('Features:', X.columns[model.support_])

Features: Index(['Dexa_During_Rx', 'Injectable_Experience_During_Rx',
       'Comorb_Encounter_For_Screening_For_Malignant_Neoplasms',
       'Comorb_Encounter_For_Immunization',
       'Comorb_Long_Term_Current_Drug_Therapy',
       'Concom_Systemic_Corticosteroids_Plain', 'Risk_Recurring_Falls'],
      dtype='object')


In [70]:
predictions = model.predict(X_train)

/Users/juliadonato/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but GeneticSelectionCV was fitted without feature names
  warnings.warn(


In [71]:
from sklearn.metrics import classification_report,confusion_matrix

In [72]:
print(classification_report(y_test,predictions))

ValueError: Found input variables with inconsistent numbers of samples: [1028, 2396]

In [43]:
print(confusion_matrix(y_test,predictions))

[[585  53]
 [169 221]]
